<a href="https://colab.research.google.com/github/eroombob/Pytorch-Tutorial-Youtube/blob/main/Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ultralytics
import torch
import torch.nn.utils.prune as prune
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [7]:
def prune_model(model, amount=0.3):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d): # Check only the convolutional layer
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

model = YOLO('yolov8s.pt')

results = model.val(data="coco8.yaml")
print(f"mAP50-95: {results.box.map}")

Ultralytics 8.3.153 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 774.6±355.2 MB/s, size: 54.0 KB)


val: Scanning /content/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.08s/it]


                   all          4         17       0.87       0.92      0.947      0.719
                person          3         10      0.855       0.59      0.707      0.379
                   dog          1          1      0.886          1      0.995      0.796
                 horse          1          2      0.794          1      0.995        0.8
              elephant          1          2          1      0.931      0.995       0.55
              umbrella          1          1      0.686          1      0.995      0.895
          potted plant          1          1          1          1      0.995      0.895
Speed: 5.2ms preprocess, 747.7ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to runs/detect/val3
mAP50-95: 0.7193132380369528


In [8]:
torch_model = model.model

print(torch_model)


print("Pruning model...")
pruned_torch_model = prune_model(torch_model, amount=0.1)
print("Model pruned.")


DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): ModuleList(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU(inplace=True)
          )
        )
      )
    )
    (3): Conv(
      (conv): Conv2d(64

In [11]:
model.model = pruned_torch_model

print("Saving pruned model...")

model.save('yolov8s_trained_pruned.pt')

print("Pruned model saved.")

Saving pruned model...
Pruned model saved.


In [12]:
model = YOLO('yolov8s_trained_pruned.pt')
results = model.val(data="coco8.yaml")
print(f"mAP50-95: {results.box.map}")


Ultralytics 8.3.153 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 956.5±253.3 MB/s, size: 54.0 KB)


val: Scanning /content/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.10s/it]


                   all          4         17      0.505      0.917      0.832      0.604
                person          3         10      0.192        0.5       0.35       0.17
                   dog          1          1       0.89          1      0.995      0.895
                 horse          1          2      0.537          1      0.995      0.764
              elephant          1          2      0.306          1      0.662      0.252
              umbrella          1          1      0.605          1      0.995      0.895
          potted plant          1          1        0.5          1      0.995      0.647
Speed: 12.7ms preprocess, 997.4ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/detect/val4
mAP50-95: 0.6038920207937098
